# What's new in scipp

This page highlights feature additions and discusses major changes from recent releases.
For a full list of changes see the [Release Notes](https://scipp.github.io/about/release-notes.html).

In [ ]:
import numpy as np
import scipp as sc

## General

### Unique dimensions and slicing of 1-D objects

<div class="alert alert-info">

**New in 0.9**

The new `dim` property checks whether an object is 1-D, and returns the only dimension label.
An exception is raised if the object is not 1-D.
</div>

Example:

In [ ]:
x = sc.linspace(dim='x', start=0, stop=1, num=4)
x.dim

<div class="alert alert-info">

**New in 0.11**

1-D objects can now be sliced without specifying a dimension.
</div>

Example:

In [ ]:
x[-1]

If an object is not 1-D then `DimensionError` is raised:

In [ ]:
var2d = sc.concat([x,x], 'y')
var2d[0]

### Logging support

<div class="alert alert-info">

**New in 0.9**

Scipp now provides a logger, and a pre-configured logging widget for Jupyter notebooks.
See [Logging](../reference/logging.ipynb).
    
</div>

### Slicing with stride

<div class="alert alert-info">

**New in 0.12**

Positional slicing (slicing with integer indices, as opposed to slicing with a label matching a coordinate value) now supports strides.

Negative strides are currently not supported.

</div>

Examples:

In [ ]:
y = sc.arange('y', 10)
y[::2]

In [ ]:
x = sc.linspace('x', 0.0, 1.0, num=5)
da = sc.DataArray(sc.ones(dims=['x', 'y'], shape=[4,10], unit='K'), coords={'x':x, 'y':y})
da['y', 1::2]

Slicing a dimension with a bin-edge coordinate with a stride is ill-defined and not supported:

In [ ]:
da['x', ::2]

### Unified conversion of unit and dtype

<div class="alert alert-info">

**New in 0.11**

Variables and data arrays have a new method, `to`, for conversion of dtype, unit, or both.
This can be used to replace uses of `to_unit` and `astype`.

</div>

Example:

In [ ]:
var = sc.arange(dim='x', start=0, stop=4, unit='m')
var

Use the `unit` keyword argument to convert to a different unit:

In [ ]:
var.to(unit='mm')

Use the `dtype` keyword argument to convert to a different dtype:

In [ ]:
var.to(dtype='float64')

If both `unit` and `dtype` are provided, the implementation attempts to apply the two conversions in optimal order to reduce or avoid the effect of rounding/truncation errors:

In [ ]:
var.to(dtype='float64', unit='km')

### Support for `unit=None`

<div class="alert alert-info">

**New in 0.12**

Previously scipp used `unit=sc.units.dimensionless` (or the alias `unit=sc.units.one`) for anything that does not have a unit, such as strings, booleans, or bins.
To allow for distinction of actual physically dimensionless quantities from these cases, scipp now supports variables and, by extension, data arrays that have their unit set to `None`.
    
This change is accompanied by a number of related changes:

- Creation function use a default unit if not given explicitly.
  The default for *numbers* (floating point or integer) is `sc.units.dimensionless`.
  The default for everything else, including `bool` is `None`.
- Comparison operations, which return variables with `dtype=bool`, have `unit=None`.
- A new function `index` was added, to allow for creation of 0-D variable with `unit=None`.
  This complements `scalar`, which uses the default unit (depending on the `dtype`).

</div>

Examples:

In [ ]:
print(sc.array(dims=['x'], values=[1.1,2.2,3.3]))
print(sc.array(dims=['x'], values=[1,2,3]))
print(sc.array(dims=['x'], values=[False, True, False]))
print(sc.array(dims=['x'], values=['a','b','c']))

In [ ]:
a = sc.array(dims=['x'], values=[1,2,3])
b = sc.array(dims=['x'], values=[1,3,3])
print(a == b)
print(a < b)

In [ ]:
(a == b).unit is None

For some purposes we may use a coordinate with unique integer-valued identifiers.
Since the identifiers to not have a physical meaning, we use `unit=None`.
Note that this has to be given explicitly since otherwise integers are treated as numbers, i.e., the unit would be dimensionless:

In [ ]:
da = sc.DataArray(a, coords={'id':sc.array(dims=['x'], unit=None, values=[34,21,14])})
da

The `index` function can now be used to conveniently lookup data by its identifier:

In [ ]:
da['id', sc.index(21)]

### Operations

#### Creation functions

<div class="alert alert-info">

**New in 0.11**
    
Creation functions for datetimes where added:

- Added `epoch`, `datetime` and `datetimes`.

</div>

In [ ]:
sc.datetime('now', unit='ms')

In [ ]:
times = sc.datetimes(dims=['time'], values=['2022-01-11T10:24:03', '2022-01-11T10:24:03'])
times

The new `epoch` function is useful for obtaining the time since epoch, i.e., a time difference (`dtype='int64'`) instead of a time point (`dtype='datetime64'`):

In [ ]:
times - sc.epoch(unit=times.unit)

<div class="alert alert-info">

**New in 0.12**
    
`zeros_like`, `ones_like`, `empty_like`, and `full_like` can now be used with data arrays.

</div>

Example:

In [ ]:
x = sc.linspace('x', 0.0, 1.0, num=5)
da = sc.DataArray(sc.ones(dims=['x', 'y'], shape=[4,6], unit='K'), coords={'x':x})
sc.zeros_like(da)

#### Utility methods and functions

<div class="alert alert-info">

**New in 0.12**
    
- Added `squeeze` method to remove length-1 dimensions from objects.
- Added `rename` method to rename dimensions and associated dimension-coordinates (or attributes).
  This complements `rename_dims`, which only changes dimension labels but does not rename coordinates.
- Added `midpoints` to compute bin-centers.

</div>

Example:

In [ ]:
x = sc.linspace('x', 0.0, 1.0, num=5)
da = sc.DataArray(sc.ones(dims=['x', 'y'], shape=[4,6], unit='K'), coords={'x':x})

A length-1 x-dimension...

In [ ]:
da['x', 0:1]

... can be removed with `squeeze`:

In [ ]:
da['x', 0:1].squeeze()

`squeeze` returns a new object and leaves the original unchanged.

Renaming is most convenient using keyword arguments:

In [ ]:
da.rename(x='xnew')

`rename` returns a new object and leaves the original unchanged.

`midpoints` can be used to replace a bin-edge coordinate by bin centers:

In [ ]:
da.coords['x'] = sc.midpoints(da.coords['x'])
da

### Reduction operations

#### Internal precision in summation operations

<div class="alert alert-info">

**New in 0.9**

Reduction operations such as `sum` of single-precision (`float32`) data now use double-precision (`float64`) internally to reduce the effects of rounding errors.

</div>

#### Reductions over multiple inputs using `reduce`

<div class="alert alert-info">

**New in 0.9**

The new `reduce` function can be used for reduction operations that do not operate along a dimension of a scipp object but rather across a list or tuple of multiple scipp objects.
The mechanism is a 2-step approach, with a syntax similar to `groupby`:

</div>

In [ ]:
a = sc.linspace(dim="x", start=0.0, stop=1.0, num=4)
b = sc.linspace(dim="x", start=0.2, stop=0.8, num=4)
c = sc.linspace(dim="x", start=0.2, stop=1.2, num=4)
sc.reduce([a, b, c]).sum()

In [ ]:
reducer = sc.reduce([a, b, c])
reducer.min()

In [ ]:
reducer.max()

### Shape operations

#### `concat` replacing `concatenate`

<div class="alert alert-info">

**New in 0.9**

`concat` is replacing `concatenate` (which is deprecated now and will be removed in 0.10).
It supports a list of inputs rather than just 2 inputs.

</div>

In [ ]:
a = sc.scalar(1.2)
b = sc.scalar(2.3)
c = sc.scalar(3.4)
sc.concat([a, b, c], "x")

#### `fold` supports size -1

<div class="alert alert-info">

**New in 0.12**

`fold` now accepts up to one size (or shape) entry with value `-1`.
This indicates that the size should be computed automatically based on the input size and other provided sizes.

</div>

Example:

In [ ]:
var = sc.arange('xyz', 2448)
var.fold('xyz', sizes={'x':4, 'y':4, 'z':-1})

### Vectors and matrices

#### General

<div class="alert alert-info">

**New in 0.11**
    
`scipp.spatial` has been restructured and extended:

- New data types for spatial transforms were added:
  - `vector3` (renamed from `vector3_float64`)
  - `rotation3` (3-D rotation defined using quaternion coeffiecients)
  - `translation3` (translation in 3-D)
  - `linear_transform3` (previously `matrix_3_float64`, 3-D linear transform with, e.g., rotation and scaling)
  - `affine_transform3` (affine transform in 3-D, combination of a linear transform and a translation, defined using 4x4 matrix)
- The [scipp.spatial](https://scipp.github.io/generated/modules/scipp.spatial.html) submodule was extended with a number of new creation functions, in particular for the new dtypes.
- `matrix` and `matrices` for creating "matrices" have been deprecated. Use `scipp.spatial.linear_transform` and `scipp.spatial.linear_transforms` instead.

</div>

Note that the `scipp.spatial` subpackage must be imported explicitly:

In [ ]:
from scipp import spatial
linear = spatial.linear_transform(value=[[1,0,0],[0,2,0],[0,0,3]])
linear

In [ ]:
trans = spatial.translation(value=[1,2,3], unit='m')
trans

Multiplication can be used to combine the various transforms:

In [ ]:
linear * trans

Note that in the case of `affine_transform3` the unit refers to the translation part.
A unit for the linear part is currently not supported.

## SciPy compatibility layer

<div class="alert alert-info">

**New in 0.11**
    
A number of subpackages providing wrappers for a *subset* of functions from the corresponding packages in SciPy was added:
    
- [scipp.integrate](../generated/modules/scipp.integrate.rst) providing `simpson` and `trapezoid`.
- [scipp.interpolate](../generated/modules/scipp.interpolate.rst) providing `interp1d`.
- [scipp.optimize](../generated/modules/scipp.optimize.rst) providing `curve_fit`.
- [scipp.signal](../generated/modules/scipp.signal.rst) providing `butter` and `sosfiltfilt`.

</div>

Please refer to the function documentation for working examples.

## Performance

<div class="alert alert-info">

**New in 0.12**

- `sc.bin()` is now faster when binning or grouping into thousands of bins or more.

</div>